In [3]:
import numpy as np

In [10]:
b_list= [[1,2,3,4,5],[6,7,8,9,10]]
z= np.array(b_list)


In [ ]:
print(z.shape)

In [ ]:
arr = np.array([[1,2,3],[4,5,6]])
arr

In [ ]:
list = [1,2,3,4]

arr=  np.array(list)

arr.shape

In [ ]:
import os
import re
paragraphs = []

for filename in os.listdir(r"D:\UDE\6th Semester\MEMS\MEWS Data\MEWS_Essays\MEWS_Essays\Essays_all\TestingEssays"):
   with open(os.path.join(r"D:\UDE\6th Semester\MEMS\MEWS Data\MEWS_Essays\MEWS_Essays\Essays_all\TestingEssays", filename)) as f:
       text = f.read()
       #text = text.replace("ï»¿","")
       #paragraphs = text.split('\n\n')
       for line in text.split('\n'):
         print ("This is " + line + " !")
       y = open(os.path.join(r"D:\UDE\6th Semester\MEMS\MEWS Data\MEWS_Essays\MEWS_Essays\Essays_all\TestingEssays", filename), 'r')
       paragraph = 0
       lines = y.readlines()
       for idx, line in enumerate(lines):
         if not line == '\n':
             m = re.search(r'\w', line)
             str = m.group(0)
         try:
            if line == '\n' and str in lines[idx-1]:
             paragraph +=1
         except:
             pass
       if lines[-1] != '\n': # if the last line is not a new line, count a paragraph.
        paragraph +=1


# for para in paragraphs:
#     print(para)
#     print("###########################")

print (paragraph)

In [ ]:
import re
f = open('../test.txt', 'r')

paragraph = 0

lines = f.readlines()

#print(lines)

for idx, line in enumerate(lines):
    print(line)
    if not line == '\n':
        m = re.search(r'\w', line)
        str = m.group(0)

    try:
        # if the line is a newline, and the previous line has a str in it, then
        # count it as a paragraph.
        if line == '\n' and str in lines[idx-1]:
            paragraph +=1
    except:
        pass

if lines[-1] != '\n': # if the last line is not a new line, count a paragraph.
    paragraph +=1

print (paragraph)

In [ ]:
f = open('../test.txt', 'r')
text = f.read()  #\n\n denotes there is a blank line in between paragraphs.

text = text.split("\n\n")

for inp in text:
    print(inp)
    print("########################")

print(len(text))

### structuring words

In [ ]:
import os
import pandas as pd

listStructureWords =  []

df = pd.read_excel("structureWords.xlsx")
listStructureWords = df.iloc[:,0].tolist()

print(listStructureWords)

In [ ]:
import re

regex = r"\b(?:{})\b".format("|".join(listStructureWords))

s = """First of all,

########### I would say #####

 that it would be difficult to stop televison advertising which is directed toward young children in the ages from two to five.
"Televison advertising could be helpful especialy for parents if they don´t have an idea concerning to give a present to their child.
"But
 #########   on the other hand   ###############


 , watching telivison advertising in these ages can lead to the missing ability to appreciate things which are advertised in the TV

############## , such as #############

  toys or electronic devises to play with. If they always want more toys, the parents maybe will follow their wishes to make their children happy and make these wishes come true, which can lead to the missing abilities mentioned before. They also wouldn´t know the value of these things.
"My opinion to this topic is, that televsion  advertising directed to young children should be stopped. Furthermore, the parents have to have an eye on their children if they watch TV. Watching TV in these ages can also be discussed, whether it´s good for them."""

res = [r.strip() for r in re.split(regex, s)]
for s in res:
    print(s)
    print("#################")

### testing on all the data.

In [1]:
import os
from nltk import tokenize
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_samples
import pandas as pd
from sentence_transformers import SentenceTransformer,util
import re

In [4]:
model = SentenceTransformer('all-mpnet-base-v2', device="cuda")

In [5]:
listStructureWords =  []

dfSrtucturingWords = pd.read_excel("structureWords.xlsx")

listStructureWords = dfSrtucturingWords.iloc[:,0].tolist()

regex = r"\b(?:{})\b".format("|".join(listStructureWords))
sentences = []

df_fileName = pd.DataFrame({})

for filename in os.listdir(r"D:\UDE\6th Semester\MEMS\MEWS Data\MEWS_Essays\MEWS_Essays\Essays_all\10 essays for essays reference\2"):
   with open(os.path.join(r"D:\UDE\6th Semester\MEMS\MEWS Data\MEWS_Essays\MEWS_Essays\Essays_all\10 essays for essays reference\2", filename)) as f:
       text = f.read()
       text = text.replace("ï»¿","")
       sents = re.split(regex, text)
       sents = tokenize.sent_tokenize(text)
       for s in sents:
           sentses = tokenize.sent_tokenize(s)
           if (s.isspace() or len(s) ==0):
               continue
           s = s.lower()
           if (len(s.split()) <5):
               continue
           for ss in sentses:
                 if (len(ss.split()) >5):
                     sentences.append(ss)

In [6]:
corpus_embeddings = model.encode(sentences, show_progress_bar =True, device="cuda")

def mbkmeans_clusters(X, k, mb=500, print_silhouette_values=False):
    """Generate clusters.

    Args:
        X: Matrix of features.
        k: Number of clusters.
        mb: Size of mini-batches. Defaults to 500.
        print_silhouette_values: Print silhouette values per cluster.

    Returns:
        Trained clustering model and labels based on X.
    """
    km = MiniBatchKMeans(n_clusters=k, batch_size=mb).fit(X)
    print(f"For n_clusters = {k}")
    print(f"Silhouette coefficient: {silhouette_score(X, km.labels_):0.2f}")
    print(f"Inertia:{km.inertia_}")

    if print_silhouette_values:
        sample_silhouette_values = silhouette_samples(X, km.labels_)
        print(f"Silhouette values:")
        silhouette_values = []
        for i in range(k):
            cluster_silhouette_values = sample_silhouette_values[km.labels_ == i]
            silhouette_values.append(
                (
                    i,
                    cluster_silhouette_values.shape[0],
                    cluster_silhouette_values.mean(),
                    cluster_silhouette_values.min(),
                    cluster_silhouette_values.max(),
                )
            )
        silhouette_values = sorted(
            silhouette_values, key=lambda tup: tup[2], reverse=True
        )
        for s in silhouette_values:
            print(
                f"    Cluster {s[0]}: Size:{s[1]} | Avg:{s[2]:.2f} | Min:{s[3]:.2f} | Max: {s[4]:.2f}"
            )
    return km, km.labels_

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
k = 19

clustering, cluster_labels = mbkmeans_clusters(X=corpus_embeddings, k=k, print_silhouette_values=True)

df_clusters = pd.DataFrame({
    "text": sentences,
    "cluster": cluster_labels
})

df_clusters.to_excel("YourNewExcel.xlsx")

In [ ]:
df_test = pd.DataFrame({})

df_ref = pd.read_excel(r"D:\USERS-Load\PycharmProjects\pythonProject\CVS Files\01.08.22\new Gold standards.xlsx")

def_clusters = pd.read_excel(r"YourNewExcel.xlsx")

listRef = df_ref['Gold standard']

listClusters = def_clusters['cluster']
listNewSentences = def_clusters['text']
listembReferences = []

for ref in listRef:
    listembReferences.append(model.encode(ref))

cosListRefToClusters = []
mm=0
for emb in corpus_embeddings:
    listMax = []
    for embref in listembReferences:
        cos_sim = util.cos_sim(emb, embref)
        listMax.append(cos_sim)
    index_max = np.argmax(listMax) #this is the right reference cluster
    cosListRefToClusters.append(index_max)
    df_test = df_test.append({
        "ID" : mm,
        "text": listNewSentences[mm],
        "cluster": listClusters[mm],
        "cos Sim": index_max
              },ignore_index=True)
    mm = mm+1


df_test.to_excel("YourNewExcel.xlsx");

In [ ]:
tpCount= 0
tnCount  = 0

fnCount = 0
fpcount = 0


dfPairEvaluation = pd.read_excel(r"YourNewExcel.xlsx")

iteration = dfPairEvaluation.count()['ID']

for i in range(0,iteration):
    for j in range (i+1, iteration):


        clusterSystem1 = dfPairEvaluation.query(f"ID == {i}")['cluster'].iloc[0]
        clusterGold1 = dfPairEvaluation.query(f"ID == {i}")['cos Sim'].iloc[0]
        clusterSystem2 = dfPairEvaluation.query(f"ID == {j}")['cluster'].iloc[0]
        clusterGold2 = dfPairEvaluation.query(f"ID == {j}")['cos Sim'].iloc[0]


        if (clusterSystem1 == clusterSystem2):

            if (clusterGold1 ==clusterGold2): # it is a true positive
                tpCount = tpCount+ 1

            else:
                fnCount = fnCount + 1   # it is a false negative
        else:
            if (clusterGold1 !=clusterGold2):
                tnCount = tnCount + 1  # it is a true negative
            else:
                fpcount = fpcount+1   # it is a false positive.

print("the value of tp is", tpCount)
print("the value of tn is", tnCount)
print("the value of fn is", fnCount)
print("the value of fb is", fpcount)

accuracy = (tpCount + tnCount)/(tpCount +tnCount + fpcount+ fnCount)

print("The accuracy is ",accuracy*100)  # the value is higher than the other accuracy.

percision = tpCount / (tpCount + fpcount)

print("The percision is ",percision*100)

recall = tpCount / (tpCount + fnCount)

print("The recall is ",recall*100)

In [ ]:
from collections import defaultdict


dfPurity = pd.read_excel(r"YourNewExcel.xlsx")
d = defaultdict(list)

for j in range(0, k-1):
    mylist = dfPurity.query(f"cluster == {j}")['cos Sim']
    d[j].extend(mylist)

rightCounter = 0

for key, value in d.items():
    mostFreq = max(set(value), key = value.count)
    rightCounter = rightCounter + value.count(mostFreq)
    #print(key , value.count(mostFreq))

total = dfPurity.count()['ID']

print("purity = ", (rightCounter/total)*100)

### calculate the count of each culture for the first research question

In [ ]:
df_numbersAuto = pd.read_excel(r"D:\USERS-Load\PycharmProjects\pythonProject\sbert\GermanExcelResearchQ1-Normalised-AD.xlsx")
df_test = pd.DataFrame({})

for j in range(0, 19):
    listClustersIDRef = df_numbersAuto.query(f"cosSim == {j}")['ID']
    df_test = df_test.append({
        "Cluster Reference" : j,
        "count": len(listClustersIDRef)
              },ignore_index=True)

df_test.to_excel("GermanExcelResearchQ1 - AD - Normalised-Counts.xlsx")

### calculate the purity and the accuracy for the hand made 10 essays

In [ ]:
tpCount= 0
tnCount  = 0

fnCount = 0
fpcount = 0


dfPairEvaluation = pd.read_csv(r"D:\USERS-Load\PycharmProjects\pythonProject\CSV Files\Evaluation\Gold vs Clusters - Word2Vec.csv")

iteration = dfPairEvaluation.count()['ID']

for i in range(0,iteration):
    for j in range (i+1, iteration):


        clusterSystem1 = dfPairEvaluation.query(f"ID == {i}")['cluster'].iloc[0]
        clusterGold1 = dfPairEvaluation.query(f"ID == {i}")['GOLDStandards'].iloc[0]
        clusterSystem2 = dfPairEvaluation.query(f"ID == {j}")['cluster'].iloc[0]
        clusterGold2 = dfPairEvaluation.query(f"ID == {j}")['GOLDStandards'].iloc[0]


        if (clusterSystem1 == clusterSystem2):

            if (clusterGold1 ==clusterGold2): # it is a true positive
                tpCount = tpCount+ 1

            else:
                fnCount = fnCount + 1   # it is a false negative
        else:
            if (clusterGold1 !=clusterGold2):
                tnCount = tnCount + 1  # it is a true negative
            else:
                fpcount = fpcount+1   # it is a false positive.

print("the value of tp is", tpCount)
print("the value of tn is", tnCount)
print("the value of fn is", fnCount)
print("the value of fb is", fpcount)

accuracy = (tpCount + tnCount)/(tpCount +tnCount + fpcount+ fnCount)

print("The accuracy is ",accuracy*100)  # the value is higher than the other accuracy.

percision = tpCount / (tpCount + fpcount)

print("The percision is ",percision*100)

recall = tpCount / (tpCount + fnCount)

print("The recall is ",recall*100)

##############################################################################################################

from collections import defaultdict
dfPurity = pd.read_csv(r"D:\USERS-Load\PycharmProjects\pythonProject\CSV Files\Evaluation\Gold vs Clusters - Word2Vec.csv")
d = defaultdict(list)

for j in range(0, 19):
    if (j==3 or j==9):
        continue
    mylist = dfPurity.query(f"cluster == {j}")['GOLDStandards']
    d[j].extend(mylist)

rightCounter = 0

for key, value in d.items():
    mostFreq = max(set(value), key = value.count)
    rightCounter = rightCounter + value.count(mostFreq)
    #print(key , value.count(mostFreq))

total = dfPurity.count()['ID']

print("purity = ", (rightCounter/total)*100)

In [31]:
df = pd.read_csv("C:/Users/Ahmed/Downloads/archive/df_full_premierleague.csv")

In [ ]:
df.head()

In [ ]:
df.drop(df.iloc[:, 37:114], inplace=True, axis=1)

df

In [38]:
df.to_csv("newCsv.csv",index=False)

In [36]:
df = df.reset_index(drop=True)

In [ ]:
df

### Test all the combinataion of vectorizer and the splitting

In [ ]:
from sentence_transformers import SentenceTransformer
from Utility.ThesisUtility import mbkmeans_clusters,writeClustersToExcel,getSentenceUsingStructringWords,printPairsEvaluation,printPurity,iterateAndAssignGoldStandard

dataFilePath = "../Dataset/Essays_all/10 essays for essays reference - ads/1"
evaluationFilePath = "../CSV Files/Clusters Evaluation results/GOLD vs Clusters - SBERT.xlsx"
k = 19 # the value for the K-Means
modelFileName = "SbertSentences.txt"
clusterExcelSheetName = "clusteredArgument- Sbert - "
#defining the model
model = SentenceTransformer('all-mpnet-base-v2', device="cuda")

## Reading the gold standards and vectorizing them
df_ref = pd.read_excel("../CSV Files/Goldstandards/AD/Goldstandards.xlsx")
listRef = df_ref['Gold standard']
refs = model.encode(listRef)

sentences = getSentenceUsingStructringWords (dataFilePath,False)
corpus_embeddings = model.encode(sentences, show_progress_bar =True, device="cuda")

iterateAndAssignGoldStandard(corpus_embeddings,sentences,refs,"Sbert With Centriods.xlsx",False)

### Get the avg words per sentence for each method

In [2]:
from Utility.ThesisUtility import getSentcesAndTokens, avg_words_in_list

In [1]:
from Utility.ThesisUtility import getSentcesAndTokens, avg_words_in_list,getSentenceUsingStructringWords

dataFilePath = "../Dataset/Essays_all/TeachersMerged T1+T2"

NormalSentences = getSentcesAndTokens(dataFilePath,False)
Segments = getSentenceUsingStructringWords (dataFilePath,False)

avgNomrmalSentences = avg_words_in_list(NormalSentences)
avgSegments = avg_words_in_list(Segments)

print("number of NormalSentences", len(NormalSentences))
print("number of Segments", len(Segments))

print("Avg number of words NormalSentences", avgNomrmalSentences)
print("Avg number of words Segments", avgSegments)

number of NormalSentences 34692
number of Segments 33747
Avg number of words NormalSentences 20.970973135016717
Avg number of words Segments 21.447891664444246


In [ ]:
import scipy
from sklearn.metrics import cohen_kappa_score, confusion_matrix

df = pd.read_excel("C:/Users/Ahmed/Downloads/nlp/ToBeAnnotatedSentences - merged.xlsx")
y1 = df.query(f"Final_1st != {-1}")['Final_1st']
y2 = df.query(f"Final_2nd != {-1}")['Final_2nd']

print ("Kappa - ", cohen_kappa_score(y1[0:235],y2[0:235],weights="linear"))
print ("Pearson - ", scipy.stats.pearsonr(y1[0:235], y2[0:235]))
cm  = confusion_matrix(y1[0:235], y2[0:235])

In [ ]:
y1[0:235]

In [35]:
from matplotlib import pyplot as plt
import seaborn as sns

# Change figure size and increase dpi for better resolution
plt.figure(figsize=(20,10), dpi=1000)
# Scale up the size of all text
sns.set(font_scale = 1.1)

# Plot Confusion Matrix using Seaborn heatmap()
# Parameters:
# first param - confusion matrix in array format
# annot = True: show the numbers in each heatmap cell
# fmt = 'd': show numbers as integers.
ax = sns.heatmap(cm, annot=True, fmt='d', )

# set x-axis label and ticks.
ax.set_xlabel("1st annotator", fontsize=14, labelpad=20)

# set y-axis label and ticks
ax.set_ylabel("2nd annotator", fontsize=14, labelpad=20)

# set plot title
ax.set_title("Confusion Matrix between 1st and 2nd annotator", fontsize=14, pad=20)

plt.show()